# 2 Creation of splits
This notebook presents how the splits for the models and the dataset version are handlet inside the plankton classifier pipeline.

## 1 Challenges and requiremnts of the split creation

In order to ensure the reproducibility of old splits and to provide enough felxibility to try new split, a own modul was coded that could fulfill following points:

- Reproducibility of old splits
- Creation of new splits based on past data versions
- Consideration of an OOD data set
- Simple to implement extensions with own split methods


This notebook shows how the split processing is handled, how to use it and how to register a new split version.

### 1.1 Requires

The notebook requires that all images are saved in the corresponding class folder, irrespective of the dataset version. Additionally, an overview data frame with the splits of the ZooLake version 1 and 2 must be provided. This can be achieved by utilising the notebook `1_data_set_overview.ipynb`, which is included in the notebook directory.

### 1.2 Preparations

The preparatios includes the navigating to the right dir level, installing the needed package and loading the overview dataframe


In [9]:
import logging


# Clear existing handlers
logger = logging.getLogger()
if logger.hasHandlers():
    logger.handlers.clear()

logger.setLevel(logging.DEBUG)

# Formatter and StreamHandler for the Notebook
handler = logging.StreamHandler()

# set format with file, line number, function name
handler.setFormatter(logging.Formatter('%(asctime)s - %(name)s - %(levelname)s - %(message)s'))
logger.addHandler(handler)

# Example log messages
logger.debug("Example debug message")
logger.info("Example info message")
logger.warning("Example warning message")

2024-12-20 12:27:09,475 - root - DEBUG - Example debug message
2024-12-20 12:27:09,476 - root - INFO - Example info message
2024-12-20 12:27:09,477 - root - WARNING - Example warning message


In [10]:
# preparations
import os 

import pandas as pd

def check_current_work_dir():
    if not os.path.isfile("setup.py") or  os.path.basename(os.getcwd()).endswith('notebooks'):
        print("Changing the current directory to the parent directory containing the setup.py file")

        # move one folder up
        os.chdir("..")
        print(f"New current directory: {os.getcwd()}, it will remain this working directory for the rest of the notebook")

    if not os.path.isfile("setup.py"):
        raise Exception("setup.py not found in the current directory")

check_current_work_dir()

check_current_work_dir()
# installation of the package 
# "%"  makes the installation from a notebook cell out possible
# and "." since the setup.py is in the current directory
%pip install .


Processing c:\repos\plankton_classifier
  Installing build dependencies: started
  Installing build dependencies: finished with status 'done'
  Getting requirements to build wheel: started
  Getting requirements to build wheel: finished with status 'done'
  Preparing metadata (pyproject.toml): started
  Preparing metadata (pyproject.toml): finished with status 'done'
  Created wheel for lit_ecology_classifier: filename=lit_ecology_classifier-2.0-py3-none-any.whl size=101138 sha256=f83ead0997f5292e91ccd02c856f8a42316b9f672de6826ee4d242f17b304c97
  Stored in directory: C:\Users\ruizjuan\AppData\Local\Temp\pip-ephem-wheel-cache-7dgvgpnf\wheels\ef\4e\b8\2be6c16208ad63911f0ceed2b4bee5a863a52ea77b22187903
Successfully built lit_ecology_classifier
  Attempting uninstall: lit_ecology_classifier
    Found existing installation: lit_ecology_classifier 2.0
    Uninstalling lit_ecology_classifier-2.0:
      Successfully uninstalled lit_ecology_classifier-2.0
Note: you may need to restart the kerne


[notice] A new release of pip is available: 23.2.1 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [11]:
# load the overview data
path = os.path.join("data", "interim",  "overview.csv")
df = pd.read_csv( filepath_or_buffer= path)
df.head()

,image,class,sha256,date,OOD_v2,version_1,version_2,train_v1,test_v1,val_v1,train_v2,test_v2,val_v2
0,SPC-EAWAG-0P5X-1570543372901157-3725350526242-...,aphanizomenon,6fb0b3fa4b36614703ee1abdcf8efba4cd936982ca5fb6...,2019-10-08 14:02:52+00:00,False,True,True,True,False,False,True,False,False
1,SPC-EAWAG-0P5X-1570543374882008-3725352526408-...,aphanizomenon,09e4aa12fdc992bbd840b7913f6f35394637bc2135c49f...,2019-10-08 14:02:54+00:00,False,True,True,True,False,False,False,True,False
2,SPC-EAWAG-0P5X-1589472012505862-10217420880920...,aphanizomenon,1ace5cdd5a68e8cd5fa703c92ac7c6e6b1d362b517132f...,2020-05-14 16:00:12+00:00,False,True,True,True,False,False,True,False,False
3,SPC-EAWAG-0P5X-1589472120505648-10217528889899...,aphanizomenon,f9a38d8538b1ac64383199851c61ad2f7f784e430086ea...,2020-05-14 16:02:00+00:00,False,True,True,True,False,False,True,False,False
4,SPC-EAWAG-0P5X-1589472215513831-10217623897796...,aphanizomenon,9cfb8f3f9d36cb50c32bedc72724092a7a01576ccb8529...,2020-05-14 16:03:35+00:00,False,True,True,False,False,True,True,False,False


In [12]:
# Transformation to a tidy format
df = df.filter(regex="train|test|val|image|class|sha256|OOD")

# Melt test, train, and val columns into one column per row and version
df_melted = df.filter(regex="v1|v2|image|class|sha256").melt(
    id_vars=['image', 'class', 'sha256'],
    var_name='split'
)

# Extract 'version' and 'split' from the 'split' column using vectorized string methods
df_melted["version"] = df_melted["split"].str.split('_').str[1]
df_melted["split"] = df_melted["split"].str.split('_').str[0]

# Keep only rows where 'value' is 1
df_melted = df_melted[df_melted['value'] == 1]

# Replace version labels for consistency
df_melted["version"] = df_melted["version"].replace({"v1": "1", "v2": "2"})

# Drop the 'value' column as it's no longer needed
df_melted = df_melted.drop(columns="value")

# Display the first few rows of the transformed dataframe
df_melted.head()


,image,class,sha256,split,version
29640,SPC-EAWAG-0P5X-1624734366033156-29241721188723...,aphanizomenon,f918287e56745c04bda26e4e41c7410829460f1e00c9b8...,OOD,2
29641,SPC-EAWAG-0P5X-1624662155318571-29169511575069...,asplanchna,a09365d3b45b8d69976b0796f885d51c8763b7e035def9...,OOD,2
29642,SPC-EAWAG-0P5X-1624662157341137-29169513575235...,asplanchna,a4ca7d1c4daac22313245706347e3544fa90722273472c...,OOD,2
29643,SPC-EAWAG-0P5X-1624662354355777-29169710591612...,asplanchna,55df99360fdc74e9485846e4d836232df919e832e23a1e...,OOD,2
29644,SPC-EAWAG-0P5X-1624662437359090-29169793598512...,asplanchna,1610bc0981eb9b8543fa0a4e8b6afaa1a22788b6fdbcbc...,OOD,2



## 2 Split Overview

The split overview was created with the ambition to keep informations and track of the used split. It includes following columns:

$$
\small

\begin{array}{c}
\textbf{Description of the split overview dataframe}\\
\newline
\begin{array}{|l|l|}
\hline
\textbf{Column name} & \textbf{Description} \\
\hline
\text{dataset version} & \text{Version of the Dataset used for the split} \\
\text{OOD} & \text{OOD version included in the split} \\
\text{split strategy} & \text{Used split strategy} \\
\text{combined split hash} & \text{Hash value to identfiy and check the split} \\
\text{Description} & \text{Place to add a short description of the split} \\
\hline
\end{array}
\end{array}

$$

\
The current status of the split overview can be found in the folder "interim\UsedSplit\". Nevertheless, the intention is to store the overview split in a database in the near future.



In [13]:
# loading of the split overview 
path = os.path.join("data", "interim", "UsedSplits", "split_overview.csv")

types = {
    "dataset_version": "str",
    "OOD": "str",
    "split_strategy": "str",
    "combined_split_hash": "str",
    "fescription": "str"
}

split_overview = pd.read_csv(filepath_or_buffer= path, index_col=False, dtype=types)
split_overview.head()

,dataset_version,OOD,split_strategy,filter_strategy,combined_split_hash,description
0,1,,Unknown,PlanktonFilter,7ae8bedcd1f7b93380ada9d97df367f75e4ff22c0f9214...,Split used for Deep Learning Classification of...
1,2,OOD_2,Unknown,PlanktonFilter,7ac1342e84ca156574ef657e342945eeee398dc01c0563...,Split used for Producing Plankton Classifiers ...


## 3 Split hashes and comnined split hash

In order to not only store information but also to verify whether the reconstructed splits align with the original ones,  the property of hashing was again utilised based on the  images hashesh.

### 3.1 Split hashes

The split hashes are calculated based on a sorted join of each hash within each unique value of the column "split".  This is shown for the first Zoolake version below.


In [14]:
# example of the split hashes 

from lit_ecology_classifier.helpers.hashing import HashGenerator

# filter to calculate the hashes for the first version
df_v1 = df_melted[df_melted["version"] == "1"]

# generate the hashes
hashes_v1 = HashGenerator().generate_hash_dict_from_split(df_v1, col_to_hash ="sha256", group_by_col= "split")

hashes_v1

{'test': '3dc0e5aadb042c37d8e52908b23c9b0af83e2497109e7e1c5a25d2b65c5e14be',
 'train': 'bc9bb5d05fbdd28547737c9953d10fa3f584e9a832dc7e2c75d1b6751f5a2024',
 'val': '1d24484776f9579e8bdd468c884dd4df15eb4429ea89d136fed636245d8c49e9'}

As demonstrated by the output, the values of the dictionary represent the calculated hash value for each split. This makes it possible to find out if each reproduced split  correspond to the original one.

### 3.1 Combined split hashes
A value per split presents the disadvantage of complicating a direct comparison of whether the entire split is identical. For this reason, the individual hash values are sorted and hashed again. This leads to the loss of knwoing wich hash differs now. Example:

In [15]:
combined_hash = HashGenerator().sha256_from_list(hashes_v1.values())    
combined_hash

'7ae8bedcd1f7b93380ada9d97df367f75e4ff22c0f9214c00e80c52845c5eaed'

The utilisation of the images and split hashes permits the theoretical identification down to the pixel value whether the splits are identical or not.

## 4 SplitProcessor

In order to fulfil the necessary requirements to create and recreate old splits, the class `SplitProcessor` was implemented. The Split Processor provides the main functionalities to manage all aspects of the data splitting process, including the checking of existing splits, the creation of new splits and the execution of the image copying. 

The implementation of the split processor is object-oriented, employing polymorphism for the split strategy and inheritance of the `base_image_mover.py` functionalities for the copying of the images. 

At the momement, the created splits and overview are stored inside the file folder `interim\UsedSplit\`  However, it is intended that this will be replaced in the near future with a database.

### 4.1 Attributes

xyz


### 4.2 How to use


#### 4.2.1 Recreation of splits 

To recreate a used split the split strategy, dataset version and OOD Version need to be defined. 

In [16]:
# recreation of a split
from lit_ecology_classifier.splitting.split_processor import SplitProcessor

split_processor = SplitProcessor(
                                split_overview = split_overview,
                                split_folder = r"data\interim\UsedSplits",
                                image_overview= "data\interim\overview.csv",
                                split_strategy= 'Unknown',
                                filter_strategy= 'PlanktonFilter'
                                )

2024-12-20 12:27:19,862 - matplotlib - DEBUG - matplotlib data path: c:\Repos\plankton_classifier\.venv\Lib\site-packages\matplotlib\mpl-data
2024-12-20 12:27:19,866 - matplotlib - DEBUG - CONFIGDIR=C:\Users\ruizjuan\.matplotlib
2024-12-20 12:27:19,898 - matplotlib - DEBUG - interactive is False
2024-12-20 12:27:19,899 - matplotlib - DEBUG - platform is win32
2024-12-20 12:27:19,963 - matplotlib - DEBUG - CACHEDIR=C:\Users\ruizjuan\.matplotlib
2024-12-20 12:27:19,965 - matplotlib.font_manager - DEBUG - Using fontManager instance from C:\Users\ruizjuan\.matplotlib\fontlist-v390.json
2024-12-20 12:27:22,284 - lit_ecology_classifier.splitting.split_processor - DEBUG - Image overview column types: image        object
class        object
sha256       object
date         object
OOD_v2         bool
version_1      bool
version_2      bool
train_v1       bool
test_v1        bool
val_v1         bool
train_v2       bool
test_v2        bool
val_v2         bool
dtype: object
2024-12-20 12:27:22,284

In [17]:
getattr(split_processor, "split_df")

,image,class,sha256,split,version
0,SPC-EAWAG-0P5X-1570543372901157-3725350526242-...,aphanizomenon,6fb0b3fa4b36614703ee1abdcf8efba4cd936982ca5fb6...,train,1
1,SPC-EAWAG-0P5X-1570543374882008-3725352526408-...,aphanizomenon,09e4aa12fdc992bbd840b7913f6f35394637bc2135c49f...,train,1
2,SPC-EAWAG-0P5X-1589472012505862-10217420880920...,aphanizomenon,1ace5cdd5a68e8cd5fa703c92ac7c6e6b1d362b517132f...,train,1
3,SPC-EAWAG-0P5X-1589472120505648-10217528889899...,aphanizomenon,f9a38d8538b1ac64383199851c61ad2f7f784e430086ea...,train,1
4,SPC-EAWAG-0P5X-1589472588541825-10217996928805...,aphanizomenon,5ca3294d8df48501fd83731564c93442fedd1002c1b6f4...,train,1
...,...,...,...,...,...
18079,SPC-EAWAG-0P5X-1563195834608473-10101205289100...,keratella_cochlearis,225a67488e4bab2ece16cf9fc2c013524608601c7b2350...,val,1
18080,SPC-EAWAG-0P5X-1563196067636305-10101438308470...,keratella_cochlearis,64882fcbfbc4aff5835c4cd21e16a0201c1a12a51c4d98...,val,1
18081,SPC-EAWAG-0P5X-1575370922015129-8552825698030-...,keratella_cochlearis,240ef901638276c790bb83c791b48eb04ecd09b53bcc43...,val,1
18082,SPC-EAWAG-0P5X-1589537321913540-10282729292872...,keratella_cochlearis,f5edd8827d77807a30626961e984e3536f66fe30d2b0c7...,val,1


#### 4.2.2 Use of build in split strategies

In [18]:
split_processor = SplitProcessor(
                                split_strategy= 'Stratified',
                                filter_strategy= 'PlanktonFilter',
                                split_overview = split_overview, 
                                image_overview= r"data\interim\overview.csv",
                                filter_args= {"dataset_version":"1"})


2024-12-20 12:27:22,406 - lit_ecology_classifier.splitting.split_processor - DEBUG - Image overview column types: image        object
class        object
sha256       object
date         object
OOD_v2         bool
version_1      bool
version_2      bool
train_v1       bool
test_v1        bool
val_v1         bool
train_v2       bool
test_v2        bool
val_v2         bool
dtype: object
2024-12-20 12:27:22,407 - lit_ecology_classifier.splitting.split_processor - INFO - Split overview loaded.
2024-12-20 12:27:22,407 - lit_ecology_classifier.splitting.split_processor - DEBUG - Updated split args: {}
2024-12-20 12:27:22,407 - lit_ecology_classifier.splitting.split_processor - DEBUG - Updated filter args: {}
2024-12-20 12:27:22,408 - lit_ecology_classifier.splitting.split_processor - DEBUG - Class name: PlanktonFilter
2024-12-20 12:27:22,408 - lit_ecology_classifier.splitting.split_processor - DEBUG - Class name: Stratified
2024-12-20 12:27:22,409 - lit_ecology_classifier.splitting.split_pro

In [19]:
getattr(split_processor, "split_overview_df")

,dataset_version,OOD,split_strategy,filter_strategy,combined_split_hash,description
0,1,,Unknown,PlanktonFilter,7ae8bedcd1f7b93380ada9d97df367f75e4ff22c0f9214...,Split used for Deep Learning Classification of...
1,2,OOD_2,Unknown,PlanktonFilter,7ac1342e84ca156574ef657e342945eeee398dc01c0563...,Split used for Producing Plankton Classifiers ...


In [20]:
getattr(split_processor, "split_df")

,class,image,sha256,date,OOD_v2,version_1,version_2,train_v1,test_v1,val_v1,train_v2,test_v2,val_v2,class_map_x,class_map_y,split
0,aphanizomenon,SPC-EAWAG-0P5X-1570543372901157-3725350526242-...,6fb0b3fa4b36614703ee1abdcf8efba4cd936982ca5fb6...,2019-10-08 14:02:52+00:00,False,True,True,True,False,False,True,False,False,1,1,train
1,aphanizomenon,SPC-EAWAG-0P5X-1570543374882008-3725352526408-...,09e4aa12fdc992bbd840b7913f6f35394637bc2135c49f...,2019-10-08 14:02:54+00:00,False,True,True,True,False,False,False,True,False,1,1,test
2,aphanizomenon,SPC-EAWAG-0P5X-1589472012505862-10217420880920...,1ace5cdd5a68e8cd5fa703c92ac7c6e6b1d362b517132f...,2020-05-14 16:00:12+00:00,False,True,True,True,False,False,True,False,False,1,1,train
3,aphanizomenon,SPC-EAWAG-0P5X-1589472120505648-10217528889899...,f9a38d8538b1ac64383199851c61ad2f7f784e430086ea...,2020-05-14 16:02:00+00:00,False,True,True,True,False,False,True,False,False,1,1,train
4,aphanizomenon,SPC-EAWAG-0P5X-1589472215513831-10217623897796...,9cfb8f3f9d36cb50c32bedc72724092a7a01576ccb8529...,2020-05-14 16:03:35+00:00,False,True,True,False,False,True,True,False,False,1,1,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39437,unknown_plankton,SPC-EAWAG-0P5X-1628924603886864-33431897306214...,f37b8728341c8523cae1e78a3014c1b4aa0f4741553c67...,2021-08-14 07:03:23+00:00,True,False,True,False,False,False,False,False,False,35,35,train
39438,unknown_plankton,SPC-EAWAG-0P5X-1628946228440913-33453521474102...,e0753b2f321e28fb34885cb5b08e1107e943345ca014ce...,2021-08-14 13:03:48+00:00,True,False,True,False,False,False,False,False,False,35,35,train
39439,unknown_plankton,SPC-EAWAG-0P5X-1628953524258605-33460817200807...,b6060aca8743971e66fbcd835dbfaa3ccc7676ba02543d...,2021-08-14 15:05:24+00:00,True,False,True,False,False,False,False,False,False,35,35,train
39440,unknown_plankton,SPC-EAWAG-0P5X-1628960940234482-33468233086890...,f422c45d51225872818e04396413ce42ea2b391da13dd9...,2021-08-14 17:09:00+00:00,True,False,True,False,False,False,False,False,False,35,35,train


In [21]:
getattr(split_processor, "class_map")

{'aphanizomenon': 1,
 'asplanchna': 2,
 'asterionella': 3,
 'bosmina': 4,
 'brachionus': 5,
 'ceratium': 6,
 'chaoborus': 7,
 'collotheca': 8,
 'conochilus': 9,
 'copepod_skins': 10,
 'cyclops': 11,
 'daphnia': 12,
 'daphnia_skins': 13,
 'diaphanosoma': 14,
 'diatom_chain': 15,
 'dinobryon': 16,
 'dirt': 17,
 'eudiaptomus': 18,
 'filament': 19,
 'fish': 20,
 'fragilaria': 21,
 'hydra': 22,
 'kellicottia': 23,
 'keratella_cochlearis': 24,
 'keratella_quadrata': 25,
 'leptodora': 26,
 'maybe_cyano': 27,
 'nauplius': 28,
 'paradileptus': 29,
 'polyarthra': 30,
 'rotifers': 31,
 'synchaeta': 32,
 'trichocerca': 33,
 'unknown': 34,
 'unknown_plankton': 35,
 'uroglena': 36}

#### Useage with own splot strategie

In [22]:
from sklearn.model_selection import train_test_split
from lit_ecology_classifier.splitting.split_strategies.base_split_strategy import BaseSplitStrategy

class ExampleSplitProcessor(BaseSplitStrategy):

    def perform_split(self, df, y_col = "class_map"):
        X = df["image"]
        y = df[y_col]
        X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

        return {
            "train": [X_train,y_train],
            "test": [X_test, y_test]    
        }


In [23]:
split_processor = SplitProcessor(split_strategy= ExampleSplitProcessor(),
                                 filter_strategy= 'PlanktonFilter',
                                 split_overview = split_overview, 
                                 image_overview= "data\interim\overview.csv",
                                 filter_args= {"dataset_version":"2"})
    


2024-12-20 12:27:22,601 - lit_ecology_classifier.splitting.split_processor - DEBUG - Image overview column types: image        object
class        object
sha256       object
date         object
OOD_v2         bool
version_1      bool
version_2      bool
train_v1       bool
test_v1        bool
val_v1         bool
train_v2       bool
test_v2        bool
val_v2         bool
dtype: object
2024-12-20 12:27:22,601 - lit_ecology_classifier.splitting.split_processor - INFO - Split overview loaded.
2024-12-20 12:27:22,601 - lit_ecology_classifier.splitting.split_processor - DEBUG - Updated split args: {}
2024-12-20 12:27:22,601 - lit_ecology_classifier.splitting.split_processor - DEBUG - Updated filter args: {}
2024-12-20 12:27:22,601 - lit_ecology_classifier.splitting.split_processor - DEBUG - Class name: PlanktonFilter
2024-12-20 12:27:22,601 - lit_ecology_classifier.splitting.split_processor - DEBUG - Class name: ExampleSplitProcessor
2024-12-20 12:27:22,601 - lit_ecology_classifier.splittin

In [24]:
getattr(split_processor, "split_overview_df")

,dataset_version,OOD,split_strategy,filter_strategy,combined_split_hash,description
0,1,,Unknown,PlanktonFilter,7ae8bedcd1f7b93380ada9d97df367f75e4ff22c0f9214...,Split used for Deep Learning Classification of...
1,2,OOD_2,Unknown,PlanktonFilter,7ac1342e84ca156574ef657e342945eeee398dc01c0563...,Split used for Producing Plankton Classifiers ...


## 5

In [25]:
split_processor.search_splits(filter_strategy= "PlanktonFilter", split_strategy= "Stratified")

2024-12-20 12:27:22,671 - lit_ecology_classifier.splitting.split_processor - DEBUG - Class name: PlanktonFilter
2024-12-20 12:27:22,671 - lit_ecology_classifier.splitting.split_processor - DEBUG - Class name: Stratified
2024-12-20 12:27:22,671 - lit_ecology_classifier.splitting.split_processor - INFO - No existing split found, creating a new split.
2024-12-20 12:27:22,671 - lit_ecology_classifier.splitting.split_processor - INFO - Creating a new split based on the given split and filter strategy.:<class 'pandas.core.frame.DataFrame'>
2024-12-20 12:27:22,671 - lit_ecology_classifier.helpers.helpers - DEBUG - Importing the modul lit_ecology_classifier.splitting.filtering.PlanktonFilter
2024-12-20 12:27:22,671 - lit_ecology_classifier.helpers.filter - DEBUG - Filtering the dataframe based on the dataset versions: []
2024-12-20 12:27:22,671 - lit_ecology_classifier.helpers.filter - DEBUG - No dataset version provided, returning the original dataframe
2024-12-20 12:27:22,671 - lit_ecology_c

,class,image,sha256,date,OOD_v2,version_1,version_2,train_v1,test_v1,val_v1,train_v2,test_v2,val_v2,class_map_x,class_map_y,split
0,aphanizomenon,SPC-EAWAG-0P5X-1570543372901157-3725350526242-...,6fb0b3fa4b36614703ee1abdcf8efba4cd936982ca5fb6...,2019-10-08 14:02:52+00:00,False,True,True,True,False,False,True,False,False,1,1,train
1,aphanizomenon,SPC-EAWAG-0P5X-1570543374882008-3725352526408-...,09e4aa12fdc992bbd840b7913f6f35394637bc2135c49f...,2019-10-08 14:02:54+00:00,False,True,True,True,False,False,False,True,False,1,1,test
2,aphanizomenon,SPC-EAWAG-0P5X-1589472012505862-10217420880920...,1ace5cdd5a68e8cd5fa703c92ac7c6e6b1d362b517132f...,2020-05-14 16:00:12+00:00,False,True,True,True,False,False,True,False,False,1,1,train
3,aphanizomenon,SPC-EAWAG-0P5X-1589472120505648-10217528889899...,f9a38d8538b1ac64383199851c61ad2f7f784e430086ea...,2020-05-14 16:02:00+00:00,False,True,True,True,False,False,True,False,False,1,1,train
4,aphanizomenon,SPC-EAWAG-0P5X-1589472215513831-10217623897796...,9cfb8f3f9d36cb50c32bedc72724092a7a01576ccb8529...,2020-05-14 16:03:35+00:00,False,True,True,False,False,True,True,False,False,1,1,test
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
39437,unknown_plankton,SPC-EAWAG-0P5X-1628924603886864-33431897306214...,f37b8728341c8523cae1e78a3014c1b4aa0f4741553c67...,2021-08-14 07:03:23+00:00,True,False,True,False,False,False,False,False,False,35,35,train
39438,unknown_plankton,SPC-EAWAG-0P5X-1628946228440913-33453521474102...,e0753b2f321e28fb34885cb5b08e1107e943345ca014ce...,2021-08-14 13:03:48+00:00,True,False,True,False,False,False,False,False,False,35,35,train
39439,unknown_plankton,SPC-EAWAG-0P5X-1628953524258605-33460817200807...,b6060aca8743971e66fbcd835dbfaa3ccc7676ba02543d...,2021-08-14 15:05:24+00:00,True,False,True,False,False,False,False,False,False,35,35,train
39440,unknown_plankton,SPC-EAWAG-0P5X-1628960940234482-33468233086890...,f422c45d51225872818e04396413ce42ea2b391da13dd9...,2021-08-14 17:09:00+00:00,True,False,True,False,False,False,False,False,False,35,35,train
